In [3]:
import pandas as pd
import os
import pickle

In [2]:
df = pd.read_excel("../../Dataset/All_Transcriptions_WithSpeakers.xlsx")

In [3]:
df

,file_name,segment_id,segment_text,segment_start,segment_end,word,word_start,word_end,speaker
0,MagnusCarlson_542_599.mp4,0,I'm going to name a sport.,0.0,0.94,I'm,0.00,0.18,Levy Rozman
1,MagnusCarlson_542_599.mp4,0,I'm going to name a sport.,0.0,0.94,going,0.18,0.28,Levy Rozman
2,MagnusCarlson_542_599.mp4,0,I'm going to name a sport.,0.0,0.94,to,0.28,0.36,Levy Rozman
3,MagnusCarlson_542_599.mp4,0,I'm going to name a sport.,0.0,0.94,name,0.36,0.46,Levy Rozman
4,MagnusCarlson_542_599.mp4,0,I'm going to name a sport.,0.0,0.94,a,0.46,0.66,Levy Rozman
...,...,...,...,...,...,...,...,...,...
12444,StarTalk_Questions_831_5_924.mp4,37,"Yeah, there's something called subspace where ...",88.6,91.62,they,90.04,90.22,Neil deGrasse Tyson
12445,StarTalk_Questions_831_5_924.mp4,37,"Yeah, there's something called subspace where ...",88.6,91.62,can,90.22,90.36,Neil deGrasse Tyson
12446,StarTalk_Questions_831_5_924.mp4,37,"Yeah, there's something called subspace where ...",88.6,91.62,communicate,90.48,90.70,Neil deGrasse Tyson
12447,StarTalk_Questions_831_5_924.mp4,37,"Yeah, there's something called subspace where ...",88.6,91.62,basically,90.70,91.14,Neil deGrasse Tyson


In [6]:
file_name_list = df["file_name"].unique()

for file_name in file_name_list:
    temp = df[df["file_name"] == file_name].reset_index(drop=True)
    temp.to_csv(f"../../Dataset/csv/{os.path.splitext(file_name)[0]}.csv", index=False, sep="\t")

In [4]:
class VideoFile:
    def __init__(self, name, save_name, start=-1, end=-1, duration=-1, ground_truth_type="der", ground_truth_file=None):
        self.name = name
        self.save_name = save_name
        self.start = start
        self.end = end
        self.duration = duration
        self.ground_truth_type = ground_truth_type
        self.ground_truth_file = ground_truth_file

In [5]:
video_file_1 = VideoFile(
    name="MagnusCarlson_542_599",
    save_name="MagnusCarlson_542_599",
    ground_truth_type="wder",
    ground_truth_file="/vol3/sunil/my_dataset/dataset/csv/MagnusCarlson_542_599.csv"
)
video_file_2 = VideoFile(
    name="StarTalk_Questions_831_5_924",
    save_name="StarTalk_Questions_831_5_924",
    ground_truth_type="wder",
    ground_truth_file="/vol3/sunil/my_dataset/dataset/csv/StarTalk_Questions_831_5_924.csv"
)

In [43]:
ado = pickle.load(open("../../output/run_output_all_videos/audio_diarization_output.pckl", "rb"))
vdo = pickle.load(open("../../output/run_output_all_videos/video_diarization_output.pckl", "rb"))

In [29]:
def find_overlap(intervals1, intervals2):
    overlap = 0
    total_duration1 = 0
    total_duration2 = 0

    # Calculate the total duration of intervals in intervals1
    for start, end in intervals1:
        total_duration1 += end - start

    # Calculate the total duration of intervals in intervals2 and find the overlap
    for start, end in intervals2:
        total_duration2 += end - start
        for s1, e1 in intervals1:
            common_start = max(s1, start)
            common_end = min(e1, end)
            if common_start < common_end:
                overlap += common_end - common_start

    # Calculate the percentage of overlap with respect to intervals1
    if total_duration1 == 0:
        percentage_overlap1 = 0
    else:
        percentage_overlap1 = (overlap / total_duration1) * 100

    # Calculate the percentage of overlap with respect to intervals2
    if total_duration2 == 0:
        percentage_overlap2 = 0
    else:
        percentage_overlap2 = (overlap / total_duration2) * 100

    return percentage_overlap1, percentage_overlap2

def get_word_to_speaker_mapping(word_start, word_end, final_output):
    final_speaker = None
    max_overlap = 0

    for speaker_key in final_output.keys():
        word_overlap, _ = find_overlap([(word_start, word_end)], final_output[speaker_key])
        if word_overlap > max_overlap:
            final_speaker = speaker_key
            max_overlap = word_overlap

    if final_speaker is None:
        final_speaker = "Unassigned"

    return final_speaker

In [24]:
def match_output(output_df):

    assigned_speaker_list = output_df["assigned_speaker"].unique()
    speaker_match = {}
    speaker_count = {}

    for assigned_speaker in assigned_speaker_list:
        if assigned_speaker == "Unassigned":
            speaker_match[assigned_speaker] = "Unassigned"
        else:
            speaker_match[assigned_speaker] = (
                output_df[output_df["assigned_speaker"] == assigned_speaker]["speaker"]
                .value_counts()
                .idxmax()
            )
            speaker_count[assigned_speaker] = len(
                output_df[output_df["assigned_speaker"] == assigned_speaker]
            )

    temp_count = {}

    for speaker_pred, speaker_actual in speaker_match.items():
        if speaker_actual in temp_count.keys():
            temp_count[speaker_actual].append(speaker_pred)
        else:
            temp_count[speaker_actual] = [speaker_pred]

    for speaker_actual, assigned_speakers in temp_count.items():
        if len(assigned_speakers) > 1:
            max_count = 0
            max_speaker = None
            for speaker in assigned_speakers:
                if speaker_count[speaker] > max_count:
                    if max_speaker is None:
                        max_speaker = speaker
                        max_count = speaker_count[speaker]
                    else:
                        speaker_match[max_speaker] = "Unassigned"
                        max_speaker = speaker
                        max_count = speaker_count[speaker]
                else:
                    speaker_match[speaker] = "Unassigned"
            speaker_match[max_speaker] = speaker_actual

    output_df["matched_speaker"] = output_df.apply(
        lambda row: speaker_match[row["assigned_speaker"]],
        axis=1,
    )

    return output_df

In [46]:
def compute_performance(output_df):

    def speaker_match(speaker, assigned_speaker, matched_speaker):
        if assigned_speaker == "Unassigned":
            return "Missed"
        elif matched_speaker == "Unassigned" or matched_speaker != speaker:
            return "Confusion"
        else:
            return "Correct"

    output_df["matched_result"] = output_df.apply(
        lambda row: speaker_match(row["speaker"], row["assigned_speaker"], row["matched_speaker"]),
        axis=1,
    )

    return output_df["matched_result"].value_counts().to_dict()

In [47]:
file_name = video_file_2.save_name
gt_path = video_file_2.ground_truth_file

df = pd.read_csv(gt_path, sep="\t")
df["assigned_speaker"] = df.apply(
    lambda row: (
        get_word_to_speaker_mapping(
            row["word_start"],
            row["word_end"],
            vdo[file_name],
        )
        if row["word_end"] > row["word_start"]
        else get_word_to_speaker_mapping(
            row["word_start"],
            row["word_end"] + 0.01,
            vdo[file_name],
        )
    ),
    axis=1,
)

matched_df = match_output(df)
compute_performance(matched_df)

{'Correct': 190, 'Missed': 60, 'Confusion': 6}